In [1]:
import numpy as np

Подбор полинома 2 степени, неприводимого в $GF(2^4)$

tbl - таблица, элементы которой соответствуют степеням примитивного элемента $\alpha$ поля $GF(2^4)$, построенного по модулю неприводимого многочлена $x^4 + x + 1$. Таким образом $\alpha^i = tbl[i], i=0..14$ 

In [2]:
tbl = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 1, 1],
    [0, 1, 1, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 1],
    [0, 1, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 1, 1],
    [1, 1, 1, 0],
    [1, 1, 1, 1],
    [1, 1, 0, 1],
    [1, 0, 0, 1],
])

In [3]:
for i in range(len(tbl)):
    print(f'{i}\t{tbl[i].tolist()}')

0	[0, 0, 0, 1]
1	[0, 0, 1, 0]
2	[0, 1, 0, 0]
3	[1, 0, 0, 0]
4	[0, 0, 1, 1]
5	[0, 1, 1, 0]
6	[1, 1, 0, 0]
7	[1, 0, 1, 1]
8	[0, 1, 0, 1]
9	[1, 0, 1, 0]
10	[0, 1, 1, 1]
11	[1, 1, 1, 0]
12	[1, 1, 1, 1]
13	[1, 1, 0, 1]
14	[1, 0, 0, 1]


In [4]:
# Сумма произвольного количества елементов в GF(2 ^ 4)
def sum_4(*arg):
    res = arg[0].copy()
    for i in range(1, len(arg)):
        res ^= arg[i]
    return res

# получаем степень alpha, представление элемента через генератор поля
def get_deg(el):
    for i in range(len(tbl)):
        if np.all(el == tbl[i]):
            return i

# Произведение произвольного количества элементов в GF(2 ^ 4) через экспоненциальное представление
def mul_4(*arg):
    deg = 0
    for i in range(len(arg)):
        if sum(arg[i]) == 0:
            return np.zeros((4,)).astype(int)
        deg += get_deg(arg[i])
    return tbl[deg % 15]

def check_null(a):
    if np.all(a == 0):
        return True
    return False

# Деление 2 элементов в GF(2 ^ 4) через экспоненциальное представление
def div_4(a, b):
    if (all(a == 0) or all(b == 0)):
        raise ValueError
    return tbl[(get_deg(a) - get_deg(b)) % 15]

Создадим массив gf, содержащий двоичное представление всех элементов поля.

In [5]:
gf = [('0000' + bin(x)[2:])[-4:] for x in range(16)]
for i in range(len(gf)):
    gf[i] = list(map(int, list(gf[i])))

gf = np.array(gf)
gf

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1]])

Будем представлять каждый полином, как вектор кэффициентов, т.е. $a_n x^n + a_{n-1} x^{n - 1} + ... + a_1 x + a_0$ представим в виде массива $[a_n, a_{n-1}, ..., a_1, a_0]$. Так как мы ищем полином 2 степени вида $x^2 + sx + k; s, k \in GF(2^4)$, неприводимый в $GF(2^4)$, то сосздадим массив polynomials всех полиномов степени 1, которые могут являться делителями полинома $x^2 + sx + k$.

In [6]:
polynomials = []

for i in range(15):
    polynomials.append([tbl[i], np.zeros((4, )).astype(int)])

for i in range(15):
    for j in range(15):
        polynomials.append([tbl[i], tbl[j]])

polynomials = np.array(polynomials)

In [7]:
# функция деления многочлена на многочлен столбиком с коэффициентами в поле GF(2^4)
def poly_div(poly, div):
    if len(div) > len(poly):
        raise ValueError
    answ = [0] * len(poly)

    p_copy = poly.copy()
    while len(p_copy) >= len(div):
        ind = len(p_copy) - len(div)
        answ[ind] = div_4(p_copy[0], div[0])

        s = [mul_4(answ[ind], div[i]) for i in range(len(div))]
        for i in range(ind):
            s.append([0, 0, 0, 0])

        s = np.array(s)
        for i in range(len(p_copy)):
            p_copy[i] = sum_4(p_copy[i], s[i])

        i = 0
        while i < len(p_copy) and all(p_copy[i] == 0):
            i += 1

        if i == len(p_copy):
            return np.zeros((4, )).astype(int)
        else:
            p_copy = p_copy[i:]

    return p_copy

Переберем все возможные $s, k \in GF(2^4)$ и найдем подходящие полиномы вида $x^2 + sx + k$.

In [8]:
for s in tbl:
    for k in tbl:
        poly = np.array([tbl[0], s, k])
        for div in polynomials:
            if all(poly_div(poly, div).ravel() == 0):
                break
        else:
            print(f'Полином неприводимый при s = {s}   k={k}')

Полином неприводимый при s = [0 0 0 1]   k=[1 0 0 0]
Полином неприводимый при s = [0 0 0 1]   k=[1 1 0 0]
Полином неприводимый при s = [0 0 0 1]   k=[1 0 1 1]
Полином неприводимый при s = [0 0 0 1]   k=[1 0 1 0]
Полином неприводимый при s = [0 0 0 1]   k=[1 1 1 0]
Полином неприводимый при s = [0 0 0 1]   k=[1 1 1 1]
Полином неприводимый при s = [0 0 0 1]   k=[1 1 0 1]
Полином неприводимый при s = [0 0 0 1]   k=[1 0 0 1]
Полином неприводимый при s = [0 0 1 0]   k=[0 0 0 1]
Полином неприводимый при s = [0 0 1 0]   k=[0 0 1 0]
Полином неприводимый при s = [0 0 1 0]   k=[0 1 1 0]
Полином неприводимый при s = [0 0 1 0]   k=[0 1 0 1]
Полином неприводимый при s = [0 0 1 0]   k=[1 0 1 0]
Полином неприводимый при s = [0 0 1 0]   k=[1 1 1 0]
Полином неприводимый при s = [0 0 1 0]   k=[1 1 0 1]
Полином неприводимый при s = [0 0 1 0]   k=[1 0 0 1]
Полином неприводимый при s = [0 1 0 0]   k=[0 0 0 1]
Полином неприводимый при s = [0 1 0 0]   k=[0 0 1 0]
Полином неприводимый при s = [0 1 0 0]   k=[0 

Например, разделим полином $x^2 + x + 1$. Можно заметить, что при деления мы получаем в некоторых случаях остаток ноль ($[0, 0, 0, 0]$), то есть полином не является неприводимым в $GF(2^4)$

In [9]:
poly = np.array([tbl[0], tbl[0], tbl[0]])
for div in polynomials:
    print(f'остаток от деления на {div.tolist()} = {poly_div(poly, div)}\n')

остаток от деления на [[0, 0, 0, 1], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[0, 0, 1, 0], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[0, 1, 0, 0], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[1, 0, 0, 0], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[0, 0, 1, 1], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[0, 1, 1, 0], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[1, 1, 0, 0], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[1, 0, 1, 1], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[0, 1, 0, 1], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[1, 0, 1, 0], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[0, 1, 1, 1], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[1, 1, 1, 0], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[1, 1, 1, 1], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[1, 1, 0, 1], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от деления на [[1, 0, 0, 1], [0, 0, 0, 0]] = [[0 0 0 1]]

остаток от

остаток от деления на [[1, 0, 1, 0], [0, 1, 0, 0]] = [[0 1 1 0]]

остаток от деления на [[1, 0, 1, 0], [1, 0, 0, 0]] = [[0 0 1 1]]

остаток от деления на [[1, 0, 1, 0], [0, 0, 1, 1]] = [0 0 0 0]

остаток от деления на [[1, 0, 1, 0], [0, 1, 1, 0]] = [[0 1 0 0]]

остаток от деления на [[1, 0, 1, 0], [1, 1, 0, 0]] = [[0 1 0 0]]

остаток от деления на [[1, 0, 1, 0], [1, 0, 1, 1]] = [[0 0 1 0]]

остаток от деления на [[1, 0, 1, 0], [0, 1, 0, 1]] = [[0 1 0 1]]

остаток от деления на [[1, 0, 1, 0], [1, 0, 1, 0]] = [[0 0 0 1]]

остаток от деления на [[1, 0, 1, 0], [0, 1, 1, 1]] = [[0 1 1 1]]

остаток от деления на [[1, 0, 1, 0], [1, 1, 1, 0]] = [[0 1 1 0]]

остаток от деления на [[1, 0, 1, 0], [1, 1, 1, 1]] = [[0 1 0 1]]

остаток от деления на [[1, 0, 1, 0], [1, 1, 0, 1]] = [[0 1 1 1]]

остаток от деления на [[1, 0, 1, 0], [1, 0, 0, 1]] = [0 0 0 0]

остаток от деления на [[0, 1, 1, 1], [0, 0, 0, 1]] = [0 0 0 0]

остаток от деления на [[0, 1, 1, 1], [0, 0, 1, 0]] = [[0 0 1 0]]

остаток от делен

Элементом поля $GF((2^4)^2)$ являются векторы размерности 2, каждый элемент которого соответствует элементу поля $GF(2^4)$ - то есть двоичный вектор размерности 4. $p(x) = a_1 x + a_0, a_i \in GF(2^4), i=1, 2$, тогда в векторном представлении $p(x) = [a_1, a_0]$.

Построим поле $GF((2^4)^2)$ по модулю неприводимого (см. выше) полинома $x^2 + \alpha^2 x + \alpha$.

In [10]:
# определим умножение и сложение в GF((2^4)^2)
def mul_8(a, b):
    q1 = sum_4(mul_4(a[0], b[0], tbl[2]), mul_4(a[0], b[1]), mul_4(a[1], b[0]))
    q0 = sum_4(mul_4(a[0], b[0], tbl[1]), mul_4(a[1], b[1]))
    return [q1, q0]

def sum_8(a, b):
    q1 = sum_4(a[0], b[0])
    q0 = sum_4(a[1], b[1])
    return [q1, q0]

Проверим, является ли корень неприводимого многочлена $x^2 + \alpha^2 x + \alpha$, по модулю которого мы строили поле $GF((2^4)^2)$, примитивным элементом этого поля.

In [11]:
lmbd = [np.array([0, 0, 0, 1]), np.array([0, 0, 0, 0])]
deg = [lmbd]

for i in range(256):
    lmbd = mul_8(lmbd, np.array([[0, 0, 0, 1], [0, 0, 0, 0]]))
    deg.append(lmbd)

In [13]:
for i in [1, 3, 5, 15, 17, 51, 85, 255]:
    print(f'lambda^{i} = {deg[i - 1]}')

lambda^1 = [array([0, 0, 0, 1]), array([0, 0, 0, 0])]
lambda^3 = [array([0, 0, 0, 1]), array([1, 0, 0, 0])]
lambda^5 = [array([0, 1, 1, 1]), array([1, 0, 1, 1])]
lambda^15 = [array([0, 0, 1, 0]), array([1, 0, 0, 1])]
lambda^17 = [array([0, 0, 0, 0]), array([0, 0, 1, 0])]
lambda^51 = [array([0, 0, 0, 0]), array([1, 0, 0, 0])]
lambda^85 = [array([0, 0, 0, 0]), array([0, 1, 1, 0])]
lambda^255 = [array([0, 0, 0, 0]), array([0, 0, 0, 1])]
